In [ ]:
from agentex import Agentex

client = Agentex(base_url="http://localhost:5003")

In [ ]:
AGENT_NAME = "ab030-tracing"

In [ ]:
# (REQUIRED) Create a new task. For Agentic agents, you must create a task for messages to be associated with.
import uuid

rpc_response = client.agents.create_task(
    agent_name=AGENT_NAME,
    params={
        "name": f"{str(uuid.uuid4())[:8]}-task",
        "params": {}
    }
)

task = rpc_response.result
print(task)

In [ ]:
# Send an event to the agent

# The response is expected to be a list of TaskMessage objects, which is a union of the following types:
# - TextContent: A message with just text content   
# - DataContent: A message with JSON-serializable data content
# - ToolRequestContent: A message with a tool request, which contains a JSON-serializable request to call a tool
# - ToolResponseContent: A message with a tool response, which contains response object from a tool call in its content

# When processing the message/send response, if you are expecting more than TextContent, such as DataContent, ToolRequestContent, or ToolResponseContent, you can process them as well

rpc_response = client.agents.send_event(
    agent_name=AGENT_NAME,
    params={
        "content": {"type": "text", "author": "user", "content": "Hello what can you do?"},
        "task_id": task.id,
    }
)

event = rpc_response.result
print(event)

In [ ]:
# Subscribe to the async task messages produced by the agent
from agentex.lib.utils.dev_tools import subscribe_to_async_task_messages

task_messages = subscribe_to_async_task_messages(
    client=client,
    task=task, 
    only_after_timestamp=event.created_at, 
    print_messages=True,
    rich_print=True,
    timeout=5,
)